In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras.applications.inception_v3 import preprocess_input
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras.optimizers import *
from keras.regularizers import *
from keras import initializers
from keras.applications import *

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="0,1" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

MODELS = {"InceptionResNetV2":InceptionResNetV2}

classes = ['coat_length_labels','collar_design_labels', 'lapel_design_labels',
           'neck_design_labels','neckline_design_labels',
           'pant_length_labels', 'skirt_length_labels', 
           'sleeve_length_labels']    

label_count = {'coat_length_labels':8,
               'collar_design_labels':5, 
               'lapel_design_labels':5,
               'neck_design_labels':5,
               'neckline_design_labels':10,
               'pant_length_labels':6, 
               'skirt_length_labels':6, 
               'sleeve_length_labels':9}

fai_result = []
#####################
width = 299
#####################
version = 'multi_cls'
#分配每个类读入内存的表格,用dict索引
df_train = {}
df_load = {}

#分配每个类的图像数据和标签内存,用dict索引
#X = {}
#y = {}

#for cur_class in classes:
    #df_train[cur_class] = pd.read_csv('../train/Annotations/{0}.csv'.format(cur_class), header=None)
    
    #X[cur_class] = np.zeros((len(df_train[cur_class]), width, width, 3), dtype=np.uint8)
    #y[cur_class] = np.zeros((len(df_train[cur_class]), label_count[cur_class]), dtype=np.uint8)
    
    #df_train[cur_class].columns = ['image_id', 'class', 'label']
    #df_load[cur_class] = df_train[cur_class].copy()
    #df_load[cur_class].reset_index(inplace=True)
    #del df_load[cur_class]['index']
    #样本总数 
    #print("各类样本总数{0}:{1}".format(cur_class,len(df_load[cur_class])))

Using TensorFlow backend.


In [2]:
df = pd.read_csv('../train/Annotations/train.csv', header=None)
df.columns = ['image_id', 'class', 'label']
dfl = df.copy()
dfl.reset_index(inplace=True)
del dfl['index']
nn = len(dfl)
XX = np.zeros((nn, width, width, 3), dtype=np.uint8)
yy = [np.zeros((nn, label_count[x])) for x in label_count.keys()]

for i in tqdm(range(nn)):
    #n_class = len(df_load['label'][0])
    #if len(tmp_label) == label_count[df_load['class'][i]]:
    XX[i] = cv2.resize(cv2.imread('../train/{0}'.format(dfl['image_id'][i])), (width, width))
print("图像已载入内存,开始载入转换的标签..")    
for i in tqdm(range(nn)):   
    tmp_label = dfl['label'][i]
    yy = [np.zeros((nn, label_count[x])) for x in label_count.keys()]
    if dfl['class'][i] == 'coat_length_labels':
        yy[0][i][tmp_label.find('y')] = 1   
    elif dfl['class'][i] == 'collar_design_labels':
        yy[1][i][tmp_label.find('y')] = 1
            
    elif dfl['class'][i] == 'lapel_design_labels':
        yy[2][i][tmp_label.find('y')] = 1
            
    elif dfl['class'][i] == 'neck_design_labels':
        yy[3][i][tmp_label.find('y')] = 1
            
    elif dfl['class'][i] == 'neckline_design_labels':
        yy[4][i][tmp_label.find('y')] = 1
            
    elif dfl['class'][i] == 'pant_length_labels':
        yy[5][i][tmp_label.find('y')] = 1
            
    elif dfl['class'][i] == 'skirt_length_labels':
        yy[6][i][tmp_label.find('y')] = 1
            
    else:
        yy[7][i][tmp_label.find('y')] = 1
print("载入转换的标签完毕,开始网络构建和训练..")   

  0%|          | 8/89683 [00:00<19:15, 77.61it/s]

图像已载入内存,开始载入转换的标签..


100%|██████████| 89683/89683 [07:27<00:00, 200.42it/s]

载入转换的标签完毕,开始网络构建和训练..


In [3]:
for KEY, MODLE in MODELS.items():
    #为299*299,设置如下
    ppreprocess = preprocess_input
    if KEY in ["InceptionV3","Xception", "InceptionResNetV2"]:
        width = 299
    elif KEY == "NASNetLarge":
        width = 331
    else:
        width = 224
        ppreprocess = imagenet_utils.preprocess_input 
    #定义模型
    #定义要finetune的模型结构
    cnn_model = MODLE(include_top=False, input_shape=(width, width, 3), weights='imagenet',pooling='avg')
    #输入模型网络的图片shape,如x = Input(shape=(256, 256, 3))
    inputs = Input((width, width, 3))#inputs = Input(shape=(784,))
    x = inputs
    #装入内存图片的预处理操作
    x = Lambda(ppreprocess, name='preprocessing')(x)
    #构建网络各模块逻辑连接
    x = cnn_model(x)
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    ######下面是新加的层########
    #因为设置了全局均值采样,所以没有flatten
    x = Dropout(0.5)(x)#其他形式:model.add(Dropout(0.25))
    #x = Flatten(name='flatten')(x)#其他形式:model.add(Flatten()),out = Flatten()(x)
    x = Dense(1024, activation='relu', name='fc1')(x)
    x = Dense(512, activation='relu', kernel_initializer=initializers.he_uniform(seed=None),name='fc2')(x)
    x = Dropout(0.5)(x)
    # n_class为对应属性的分类个数predictions[0]~predictions[8]
    predictions = [Dense(count, activation='softmax', name=name)(x) for name, count in label_count.items()]

    #将构建的模型网络实例化
    model = Model(inputs = inputs, outputs = predictions)
        
    #设置权参优化方法
    #optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    optimizer = SGD(lr=0.01, decay=0.01/40, momentum=0.9, nesterov=True)
    #optimizer = optimizers.Adam(lr=1e-4)
    #optimizer = Adam(lr=0.0001)
    #optimizer = 'rmsprop'
        
    #设置多GPU数据并行训练
    #多GPU训练,因为keras设计的自动保存最好模型,但是多GPU训练,其save()就没法用了,需定义各保存函数
    #model = multi_gpu_model(model, 2)  
        
    #所构建模型的编译,其中loss和metrics都可自定义,metrics=['accuracy',func自定义评价]
    model.compile(optimizer=optimizer,loss='categorical_crossentropy')

    # Callback实现学习率调整方案和保存最好模型即EarlyStopping
    #schedule = Step([20], [1e-4, 1e-6])
    #history = model.fit(X_train, Y_train,
    #                    batch_size=batch_size, nb_epoch=nb_epoch, validation_data=(X_test,Y_test),
    #                    callbacks=[schedule, keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0,save_best_only=True, mode='auto')
    #当监测值不再改善时，该回调函数将中止训练.
    #当early stop被激活（如发现loss相比上一个epoch训练没有下降），则经过patience个epoch后停止训练
    #keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')

    #设置训练完之后,最好的模型保存路径,可设置监控准确率而不是监控loss的变化,如monitor='val_acc'
    #checkpointer = ModelCheckpoint(filepath='../models/{0}/{0}_{1}_{2}.best.h5'.format(prefix, KEY, version), verbose=1, save_best_only=True)
    #训练开始,并保存训练过程的loss和acc变化,h代表history
    #model.fit([headline_data, additional_data], [labels, labels],
          #epochs=50, batch_size=32)
    print("配置完毕,开始网络训练,正在训练中...")
    h = model.fit(XX, yy, batch_size=16, epochs=5, shuffle=True)
    
    print("训练完毕!")
    #保存模型
    print("开始保存模型")
    model.save_weights('../multi_model.h5')
    
    

配置完毕,开始网络训练,正在训练中...
Epoch 1/5
89683/89683 [==============================] - 2446s 27ms/step - loss: 2.3849e-05 - coat_length_labels_loss: 0.0000e+00 - collar_design_labels_loss: 0.0000e+00 - lapel_design_labels_loss: 0.0000e+00 - neck_design_labels_loss: 0.0000e+00 - neckline_design_labels_loss: 0.0000e+00 - pant_length_labels_loss: 0.0000e+00 - skirt_length_labels_loss: 2.3849e-05 - sleeve_length_labels_loss: 0.0000e+00
Epoch 2/5
89683/89683 [==============================] - 2405s 27ms/step - loss: 2.3927e-05 - coat_length_labels_loss: 0.0000e+00 - collar_design_labels_loss: 0.0000e+00 - lapel_design_labels_loss: 0.0000e+00 - neck_design_labels_loss: 0.0000e+00 - neckline_design_labels_loss: 0.0000e+00 - pant_length_labels_loss: 0.0000e+00 - skirt_length_labels_loss: 2.3927e-05 - sleeve_length_labels_loss: 0.0000e+00
Epoch 3/5
89683/89683 [==============================] - 2395s 27ms/step - loss: 2.1933e-05 - coat_length_labels_loss: 0.0000e+00 - collar_design_labels_loss: 0.0000e+

In [6]:
print("训练完毕!开始网络模型在测试集测试,数据装入及测试结果保存输出...")   
#在测试集上预测并保存结果
df_test = pd.read_csv('../test/Tests/question.csv', header=None)
#定义各列名称
df_test.columns = ['image_id', 'class', 'x']
del df_test['x']
df_load = df_test.copy()
df_load.reset_index(inplace=True)
del df_load['index']
        
n = len(df_load)
X_test = np.zeros((n, width, width, 3), dtype=np.uint8)
#读取测试集图像到内存并resize
for i in range(n):
    X_test[i] = cv2.resize(cv2.imread('../test/{0}'.format(df_load['image_id'][i])), (width, width))
#以batch_size处理的速度遍历测试集,处理并预测
test_np = model.predict(X_test, batch_size=256)

nn = len(df_load)
result = []
for i in tqdm(range(nn)):
    tmp_result = ''
    if df_load['class'][i] == 'coat_length_labels':
        tmp_listdd = test_np[0][i]
        for tmp_ret in tmp_listdd:
            tmp_result += '{:.4f};'.format(tmp_ret)
        result.append(tmp_result[:-1])

    elif df_load['class'][i] == 'collar_design_labels':
        tmp_listdd = test_np[1][i]
        for tmp_ret in tmp_listdd:
            tmp_result += '{:.4f};'.format(tmp_ret)
        result.append(tmp_result[:-1])

    elif df_load['class'][i] == 'lapel_design_labels':
        tmp_listdd = test_np[2][i]
        for tmp_ret in tmp_listdd:
            tmp_result += '{:.4f};'.format(tmp_ret)
        result.append(tmp_result[:-1])

    elif df_load['class'][i] == 'neck_design_labels':
        
        tmp_listdd = test_np[3][i]
        for tmp_ret in tmp_listdd:
            tmp_result += '{:.4f};'.format(tmp_ret)
        result.append(tmp_result[:-1])


    elif df_load['class'][i] == 'neckline_design_labels':
        tmp_listdd = test_np[4][i]
        for tmp_ret in tmp_listdd:
            tmp_result += '{:.4f};'.format(tmp_ret)
        result.append(tmp_result[:-1])

    elif df_load['class'][i] == 'pant_length_labels':
        tmp_listdd = test_np[5][i]
        for tmp_ret in tmp_listdd:
            tmp_result += '{:.4f};'.format(tmp_ret)
        result.append(tmp_result[:-1])

    elif df_load['class'][i] == 'skirt_length_labels':
        tmp_listdd = test_np[6][i]
        for tmp_ret in tmp_listdd:
            tmp_result += '{:.4f};'.format(tmp_ret)
        result.append(tmp_result[:-1])

    else:
        tmp_listdd = test_np[7][i]
        for tmp_ret in tmp_listdd:
            tmp_result += '{:.4f};'.format(tmp_ret)
        result.append(tmp_result[:-1])
print("保存测试后的结果文件...")
#预测结果导入内存表格的result列
df_load['result'] = result     
df_load.to_csv('../result/{0}_{1}.csv'.format(KEY, version), header=None, index=False)
print("Complete!!!")

训练完毕!开始网络模型在测试集测试,数据装入及测试结果保存输出...


100%|██████████| 10080/10080 [00:01<00:00, 8358.25it/s]


保存测试后的结果文件...
Complete!!!
